In [3]:
import pickle 
import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity 


In [4]:
book = pd.read_csv('Books.csv', low_memory=False)
user = pd.read_csv('Users.csv')
rating = pd.read_csv('Ratings.csv')

In [5]:
rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   User-ID      1149780 non-null  int64 
 1   ISBN         1149780 non-null  object
 2   Book-Rating  1149780 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 26.3+ MB


# popularity based Recommendation

In [6]:
all_information = book.merge(rating, on='ISBN')

In [7]:
all_information.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1031136 entries, 0 to 1031135
Data columns (total 10 columns):
 #   Column               Non-Null Count    Dtype 
---  ------               --------------    ----- 
 0   ISBN                 1031136 non-null  object
 1   Book-Title           1031136 non-null  object
 2   Book-Author          1031134 non-null  object
 3   Year-Of-Publication  1031136 non-null  object
 4   Publisher            1031134 non-null  object
 5   Image-URL-S          1031136 non-null  object
 6   Image-URL-M          1031136 non-null  object
 7   Image-URL-L          1031132 non-null  object
 8   User-ID              1031136 non-null  int64 
 9   Book-Rating          1031136 non-null  int64 
dtypes: int64(2), object(8)
memory usage: 78.7+ MB


In [8]:
df = all_information.groupby('Book-Title').count()['Book-Rating'].reset_index()
df= df.rename(columns={'Book-Rating':'Num-Rating'})

In [9]:
new_df = all_information.groupby('Book-Title').sum()['Book-Rating'].reset_index()
new_df = new_df.rename(columns={'Book-Rating':'Average-Rating'})

In [10]:
df = df.merge(new_df, on ='Book-Title')

In [11]:
df = df[df['Num-Rating']>200]

In [12]:
df = df.merge(all_information,on='Book-Title').drop_duplicates('Book-Title')[['Book-Title','Book-Author','Image-URL-M','Num-Rating','Average-Rating']]


In [13]:
df['Average-Rating'] = df['Average-Rating']/df['Num-Rating']

In [14]:
df = df.head(50)

# Collaborative based Recommendation

In [15]:
col_df = all_information

In [16]:
active_user = col_df.groupby('User-ID').count()['Book-Rating']>=200
index = active_user[active_user].index
final_df = col_df[col_df['User-ID'].isin(index)]

In [17]:
famous_books = final_df.groupby('Book-Title').count()['Book-Rating']>=50
index = famous_books[famous_books].index
final_df = final_df[final_df['Book-Title'].isin(index)]

In [18]:
final_df.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,User-ID,Book-Rating
31,0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,11676,9
33,0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,36836,0
34,0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,46398,9
38,0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,113270,0
39,0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,113519,0


In [19]:
final_df.dropna()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,User-ID,Book-Rating
31,0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,11676,9
33,0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,36836,0
34,0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,46398,9
38,0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,113270,0
39,0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,113519,0
...,...,...,...,...,...,...,...,...,...,...
1028414,1878702831,Echoes,Nancy Morse,1992,Meteor Publishing Corporation,http://images.amazon.com/images/P/1878702831.0...,http://images.amazon.com/images/P/1878702831.0...,http://images.amazon.com/images/P/1878702831.0...,238781,0
1028600,0394429869,I Know Why the Caged Bird Sings,Maya Angelou,1996,Random House,http://images.amazon.com/images/P/0394429869.0...,http://images.amazon.com/images/P/0394429869.0...,http://images.amazon.com/images/P/0394429869.0...,239594,8
1028602,0449001164,The Promise,CHAIM POTOK,1997,Ballantine Books,http://images.amazon.com/images/P/0449001164.0...,http://images.amazon.com/images/P/0449001164.0...,http://images.amazon.com/images/P/0449001164.0...,239594,7
1028815,0743527631,The Pillars of the Earth,Ken Follett,2002,Encore,http://images.amazon.com/images/P/0743527631.0...,http://images.amazon.com/images/P/0743527631.0...,http://images.amazon.com/images/P/0743527631.0...,240144,0


In [20]:
final_df.shape

(58823, 10)

In [21]:
df2 = final_df.pivot_table(index='Book-Title',columns='User-ID',values='Book-Rating')

In [22]:
df2.shape

(707, 815)

In [23]:
df2.fillna(0, inplace=True)

In [24]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 707 entries, 1984 to \O\" Is for Outlaw"
Columns: 815 entries, 254 to 278418
dtypes: float64(815)
memory usage: 4.4+ MB


In [25]:
similarity_score = cosine_similarity(df2)

In [26]:
def book_recommendation(name):
    index = np.where(df2.index==name)[0][0]
    score = sorted(list(enumerate(similarity_score[index])), key=lambda x: x[1], reverse=True)[1:6]
    
    for i in score:
        print(df2.index[i[0]]) 

In [27]:
book_recommendation('1984')

Animal Farm
The Handmaid's Tale
Brave New World
The Vampire Lestat (Vampire Chronicles, Book II)
The Hours : A Novel


In [28]:
# pickle.dump(df2.to_dict(), open('df2.pkl', 'wb'))

In [29]:
# pickle.dump(similarity_score, open('similarity_score.pkl', 'wb'))

In [30]:
#pickle.dump(df, open('df.pkl', 'wb'))

In [40]:
pickle.dump(book, open('books.pkl', 'wb'))

In [39]:
print(book[book['Book-Title'] == 'Animal Farm']['Image-URL-M'].head(1).tolist())

['http://images.amazon.com/images/P/0451526341.01.MZZZZZZZ.jpg']
